# change to load data from load_data.py

use simplified data 2

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

## train 方法:

1個model train所有藥

In [30]:

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tabulate import tabulate



In [1]:
import numpy as np
import pandas as pd
import random

# custom import 
from utility_file import my_utilities as myutil
from utility_file import load_data

## Read Data

1. 決定要train 多少個藥 
2. 決定是否要delete 少於某threshold的藥
3. 用load_data裡的`load_data_for_n_med` load data

In [17]:

num_med_to_train = 3  # train 從第一隻藥數起多少隻
del_med_under_thres = 0     # 於所有medical cases中出現次數少於此數的藥->整col 刪除
                            # if set to 250 -> will leave 10 medicine

(X_np, train_X, X_val_np, val_X, y_np, train_y, y_val_np, val_y, num_col_x, num_medic) = load_data.load_data_for_n_med(num_med_to_train, del_med_thres=del_med_under_thres)

# data type check
assert(isinstance(X_np, np.ndarray))
assert(isinstance(X_val_np, np.ndarray))
assert(isinstance(y_np, np.ndarray))
assert(isinstance(y_val_np, np.ndarray))
assert(isinstance( train_X, pd.DataFrame))
assert(isinstance( val_X, pd.DataFrame))
assert(isinstance(train_y , pd.DataFrame))
assert(isinstance( val_y, pd.DataFrame))

in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).
SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 3 cols). First 10 data of y:
    麻黃  桂枝  荊芥
1    0   1   0
2    0   1   0
3    0   0   0
4    0   1   0
5    0   1   0
6    0   0   0
7    0   1   0
8    0   1   0
9    0   1   0
10   0   1   0
DeleteMedicine: shape of y is (796, 3).
type of val_idx <class 'list'>
type of train_idx <class 'li

In [18]:
# data type checking  you can run this if you suspect data type 
# else can skip this cell


# myutil.print_df(val_y, "---- y ----")
# print(val_y)
# checking
# 1. Count occurrence of na, int, float, str in y
na_count = val_y.isna().sum().sum()
str_count = val_y[val_y.map(type) == str].count().sum()
int_count = val_y[val_y.map(type) == int].count().sum()
float_count = val_y[val_y.map(type) == float].count().sum()

print(f"Number of NA values in y: {na_count}")
print(f"Number of str values in y: {str_count}")
print(f"Number of int values in y: {int_count}")
print(f"Number of float values in y: {float_count}")

Number of NA values in y: 0
Number of str values in y: 0
Number of int values in y: 477
Number of float values in y: 0


## Build and train model

In [19]:
# build and compile model

# 最後一個Dense為output layer, unit必須=藥的數量in y
# unit = number of neuron
# input_shape = input (X) 有多少個feature = # col of X
# 可任意加上layer
model = Sequential([
    Dense (units=16, input_shape=(num_col_x,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=num_medic, activation='sigmoid')
])
model.summary()

model.compile(optimizer=Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                1792      
                                                                 
 dense_4 (Dense)             (None, 32)                544       
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 2,435
Trainable params: 2,435
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 開始train model
# epochs = 要train多少次
model.fit(x=X_np, y=y_np,
    batch_size=10, epochs=10, shuffle=True, verbose=2)

Epoch 1/10


64/64 - 3s - loss: 0.4541 - accuracy: 0.3375 - 3s/epoch - 45ms/step
Epoch 2/10
64/64 - 0s - loss: 0.3847 - accuracy: 0.3155 - 164ms/epoch - 3ms/step
Epoch 3/10
64/64 - 0s - loss: 0.3453 - accuracy: 0.3516 - 168ms/epoch - 3ms/step
Epoch 4/10
64/64 - 0s - loss: 0.3148 - accuracy: 0.4537 - 163ms/epoch - 3ms/step
Epoch 5/10
64/64 - 0s - loss: 0.2819 - accuracy: 0.5432 - 169ms/epoch - 3ms/step
Epoch 6/10
64/64 - 0s - loss: 0.2554 - accuracy: 0.5479 - 160ms/epoch - 2ms/step
Epoch 7/10
64/64 - 0s - loss: 0.2281 - accuracy: 0.6405 - 162ms/epoch - 3ms/step
Epoch 8/10
64/64 - 0s - loss: 0.2072 - accuracy: 0.6327 - 157ms/epoch - 2ms/step
Epoch 9/10
64/64 - 0s - loss: 0.1888 - accuracy: 0.6232 - 152ms/epoch - 2ms/step
Epoch 10/10
64/64 - 0s - loss: 0.1580 - accuracy: 0.6468 - 152ms/epoch - 2ms/step


In [21]:
# predict
predictions = model.predict(X_val_np)


5/5 [==============================] - 0s 3ms/step


## Result Handling


In [22]:
col_names =[]
for i in  train_y.columns.tolist():
    i +='(predicted)'
    col_names.append(i) 


# df_predictions = probability of "has" that medicine (有該藥材)
df_predictions = pd.DataFrame(predictions, columns=train_y.columns.tolist())
print("predictions.shape:", predictions.shape)

yes_thres = 0.5         # threshold for summarized as "has that medicine"

# all_class_binary_prediction_df = predicted as 是否有該藥材
df_binary_prediction_all_class = df_predictions.apply(lambda x: [1 if i > yes_thres else 0 for i in x])

# pred_and_truth = predicted as 是否有該藥材, and ground truth
# pred_and_truth = pd.concat([df_binary_prediction_all_class, val_y.iloc[:, :num_medic]], axis=1)
# ^ this problematic, ignore this for now

## checking
assert(df_binary_prediction_all_class.shape==val_y.shape)

na_count = df_binary_prediction_all_class.isna().sum().sum()
str_count = df_binary_prediction_all_class[df_binary_prediction_all_class.map(type) == str].count().sum()
int_count = df_binary_prediction_all_class[df_binary_prediction_all_class.map(type) == int].count().sum()
float_count = df_binary_prediction_all_class[df_binary_prediction_all_class.map(type) == float].count().sum()

try:
    assert((float_count==0) and (na_count==0) and (str_count==0) and int_count )
except:
    print("Warning: there are wrong data type in df_binary_prediction_all_class!!!\n")
    print(f"Number of NA values in df_binary_prediction_all_class: {na_count}")
    print(f"Number of str values in df_binary_prediction_all_class: {str_count}")
    print(f"Number of int values in df_binary_prediction_all_class: {int_count}")
    print(f"Number of float values in df_binary_prediction_all_class: {float_count}")

predictions.shape: (159, 3)


In [33]:
print(df_predictions)
print("\n"+tabulate(df_predictions, headers='keys', tablefmt='psql', floatfmt=(".4f")))


foo
           麻黃        桂枝            荊芥
0    0.038679  0.748362  1.401726e-06
1    0.399784  0.951162  1.148378e-02
2    0.008941  0.954179  1.486067e-05
3    0.246979  0.272880  1.359408e-02
4    0.005566  0.999267  1.149775e-06
..        ...       ...           ...
154  0.002970  0.003206  7.908902e-06
155  0.000178  0.976729  2.750726e-09
156  0.079122  0.997741  1.073580e-05
157  0.246979  0.272880  1.359408e-02
158  0.019777  0.916090  1.715324e-04

[159 rows x 3 columns]

+-----+--------+--------+--------+
|     |   麻黃 |   桂枝 |   荊芥 |
|-----+--------+--------+--------|
|   0 | 0.0387 | 0.7484 | 0.0000 |
|   1 | 0.3998 | 0.9512 | 0.0115 |
|   2 | 0.0089 | 0.9542 | 0.0000 |
|   3 | 0.2470 | 0.2729 | 0.0136 |
|   4 | 0.0056 | 0.9993 | 0.0000 |
|   5 | 0.0885 | 0.2888 | 0.0001 |
|   6 | 0.0026 | 0.0019 | 0.0000 |
|   7 | 0.0437 | 0.5122 | 0.0006 |
|   8 | 0.0083 | 0.0015 | 0.0000 |
|   9 | 0.1757 | 0.1822 | 0.0063 |
|  10 | 0.0000 | 0.0236 | 0.0000 |
|  11 | 0.0306 | 0.8973 | 0.000

In [36]:

print(df_binary_prediction_all_class)
#myutil.print_df(df_binary_prediction_all_class, "df_binary_prediction_all_class")
print(" ----  df_binary_prediction_all_class ----")
print(tabulate(df_binary_prediction_all_class, headers='keys', tablefmt='psql', floatfmt=(".4f")))


     麻黃  桂枝  荊芥
0     0   1   0
1     0   1   0
2     0   1   0
3     0   0   0
4     0   1   0
..   ..  ..  ..
154   0   0   0
155   0   1   0
156   0   1   0
157   0   0   0
158   0   1   0

[159 rows x 3 columns]
df_binary_prediction_all_class

+-----+--------+--------+--------+
|     |   麻黃 |   桂枝 |   荊芥 |
|-----+--------+--------+--------|
|   0 | 0.0000 | 1.0000 | 0.0000 |
|   1 | 0.0000 | 1.0000 | 0.0000 |
|   2 | 0.0000 | 1.0000 | 0.0000 |
|   3 | 0.0000 | 0.0000 | 0.0000 |
|   4 | 0.0000 | 1.0000 | 0.0000 |
|   5 | 0.0000 | 0.0000 | 0.0000 |
|   6 | 0.0000 | 0.0000 | 0.0000 |
|   7 | 0.0000 | 1.0000 | 0.0000 |
|   8 | 0.0000 | 0.0000 | 0.0000 |
|   9 | 0.0000 | 0.0000 | 0.0000 |
|  10 | 0.0000 | 0.0000 | 0.0000 |
|  11 | 0.0000 | 1.0000 | 0.0000 |
|  12 | 0.0000 | 0.0000 | 0.0000 |
|  13 | 0.0000 | 0.0000 | 0.0000 |
|  14 | 0.0000 | 1.0000 | 0.0000 |
|  15 | 0.0000 | 0.0000 | 0.0000 |
|  16 | 0.0000 | 0.0000 | 0.0000 |
|  17 | 0.0000 | 0.0000 | 0.0000 |
|  18 | 0.0000 | 0.0000

In [37]:
# calculate true positive, true negative, false positive, false negative cell-by-cell 
# against the binary prediction result and val_y


df_is_correct = df_binary_prediction_all_class.copy()

# df1[col].combine(df2[col], lambda_fn )
# ^ this means combine `col` of df1 & df2 by using the lambda_fn
# in lambda pred, true, 
#       pred is cell in df_binary_prediction_all_class[col]
#       true is cell in df_is_correct[column]

for column in df_is_correct.columns:
    df_is_correct[column] = df_binary_prediction_all_class[column].combine(
            val_y[column], 
            lambda pred, true: 'TP' if (pred and true) else 
                           ('FP' if (pred and (not true)) else 
                            ('FN' if ((not pred) and true) else 
                             ('TN' if((not pred)and (not true)) else 'UN')
                             )))

counts = pd.Series(df_is_correct.values.flatten()).value_counts()  #.flatten()-> convert to 1D np arr
                                                                    # pd.Series(arr)-> convert to pd.Series
                                                                    #value_counts() -> count all unique occurence

# calculate TP, FP, FN, TN, f1 score
num_tp = counts.get('TP', 0)
num_fp = counts.get('FP', 0)
num_fn = counts.get('FN', 0)
num_tn = counts.get('TN', 0)
num_un = counts.get('UN', 0)

f1_score = 2 * num_tp / (2 * num_tp + num_fp + num_fn)
try:
    assert((num_tp+num_fp+num_fn+num_tn)== len(val_y)* num_medic)
except:
    print("wrong calculation!")
    print(f"there are {num_un} UN case!")
total_med_case =  len(val_y)* num_medic
print("*** Result ***\n")
print("total medical case: " , len(val_y))
print("f1 score:", f1_score)
print(f"TP: {num_tp}, percentage = {num_tp/total_med_case}")
print(f"FP: {num_fp}, percentage = {num_fp/total_med_case}")
print(f"FN: {num_fn}, percentage = {num_fn/total_med_case}")
print(f"TN: {num_tn}, percentage = {num_tn/total_med_case}")




*** Result ***

total medical case:  159
f1 score: 0.2880324543610548
TP: 71, percentage = 0.1488469601677149
FP: 4, percentage = 0.008385744234800839
FN: 347, percentage = 0.7274633123689728
TN: 55, percentage = 0.11530398322851153


In [26]:
# print df_is_correct
myutil.print_df(df_is_correct, "--- df is correct  -----")

--- df is correct  -----

+-----+--------+--------+--------+
|     | 麻黃   | 桂枝   | 荊芥   |
|-----+--------+--------+--------|
|   0 | FN     | TP     | FN     |
|   1 | FN     | TP     | FN     |
|   2 | FN     | TP     | FN     |
|   3 | FN     | FN     | FN     |
|   4 | FN     | TP     | FN     |
|   5 | FN     | FN     | FN     |
|   6 | TN     | TN     | TN     |
|   7 | FN     | TP     | FN     |
|   8 | FN     | FN     | FN     |
|   9 | TN     | FN     | TN     |
|  10 | TN     | FN     | TN     |
|  11 | TN     | TP     | TN     |
|  12 | FN     | FN     | FN     |
|  13 | FN     | FN     | FN     |
|  14 | TN     | TP     | TN     |
|  15 | FN     | FN     | FN     |
|  16 | FN     | FN     | FN     |
|  17 | FN     | FN     | FN     |
|  18 | TN     | FN     | TN     |
|  19 | FN     | FN     | FN     |
|  20 | FN     | FN     | FN     |
|  21 | FN     | TP     | FN     |
|  22 | TP     | TP     | FN     |
|  23 | FN     | TP     | FN     |
|  24 | TP     | TP     | TN     |


In [27]:


# 
myutil.df_to_csv(df_predictions, save_path="./result/multi_med_prediction", file_prefix="probi_each_med")
myutil.df_to_csv(df_is_correct, save_path="./result/multi_med_prediction", file_prefix="is_correct")




probi_each_med saved to ./result/multi_med_prediction/probi_each_med_5.csv
is_correct saved to ./result/multi_med_prediction/is_correct_4.csv


## Print everything

In [39]:
print(" ----- df_prediction ----")
print(df_predictions)
print(" ----- df_prediction ----")
print("\n"+tabulate(df_predictions, headers='keys', tablefmt='psql', floatfmt=(".4f")))
print("-----------------------------------------------------")
print(" ----  df_binary_prediction_all_class ----")
print(tabulate(df_binary_prediction_all_class, headers='keys', tablefmt='psql', floatfmt=(".4f")))
print("-----------------------------------------------------")
print(" ----  df_is_correct ----")
print(tabulate(df_is_correct, headers='keys', tablefmt='psql', floatfmt=(".4f")))
print("-----------------------------------------------------")

 ----- df_prediction ----
           麻黃        桂枝            荊芥
0    0.038679  0.748362  1.401726e-06
1    0.399784  0.951162  1.148378e-02
2    0.008941  0.954179  1.486067e-05
3    0.246979  0.272880  1.359408e-02
4    0.005566  0.999267  1.149775e-06
..        ...       ...           ...
154  0.002970  0.003206  7.908902e-06
155  0.000178  0.976729  2.750726e-09
156  0.079122  0.997741  1.073580e-05
157  0.246979  0.272880  1.359408e-02
158  0.019777  0.916090  1.715324e-04

[159 rows x 3 columns]
 ----- df_prediction ----

+-----+--------+--------+--------+
|     |   麻黃 |   桂枝 |   荊芥 |
|-----+--------+--------+--------|
|   0 | 0.0387 | 0.7484 | 0.0000 |
|   1 | 0.3998 | 0.9512 | 0.0115 |
|   2 | 0.0089 | 0.9542 | 0.0000 |
|   3 | 0.2470 | 0.2729 | 0.0136 |
|   4 | 0.0056 | 0.9993 | 0.0000 |
|   5 | 0.0885 | 0.2888 | 0.0001 |
|   6 | 0.0026 | 0.0019 | 0.0000 |
|   7 | 0.0437 | 0.5122 | 0.0006 |
|   8 | 0.0083 | 0.0015 | 0.0000 |
|   9 | 0.1757 | 0.1822 | 0.0063 |
|  10 | 0.0000 | 0